In [9]:
import cvxpy as cp
import numpy as np

supplies = [1200, 1200, 1200] # Delhi, Mumbai, Shanghai
capacities = [800, 1500, 800] # Jakarta, Surababaya, Bandung
demands = [1500, 1500] # Palu, Donggala

origin_2_transfer = cp.Variable((3, 3))
transfer_2_destination = cp.Variable((3, 2))

origin_2_transfer_cost = np.array([
    [10, 12, 25],
    [7, 8, 15],
    [5, 8, 10],
])
transfer_2_destination_cost = np.array([
    [20, 15],
    [15, 20],
    [10, 16],
])

constraints = [
    origin_2_transfer >= 0,
    transfer_2_destination >= 0,
    cp.sum(origin_2_transfer, axis=1) <= supplies,
    cp.sum(transfer_2_destination, axis=1) <= capacities,
    cp.sum(origin_2_transfer, axis=0) == cp.sum(transfer_2_destination, axis=1),
    cp.sum(transfer_2_destination, axis=0) >= demands,
]

objective = cp.Minimize(
    cp.multiply(origin_2_transfer, origin_2_transfer_cost).sum() +
    cp.multiply(transfer_2_destination, transfer_2_destination_cost).sum()
)

problem = cp.Problem(objective, constraints)
problem.solve()

print("Optimal value: ", problem.value)
print("Optimal origin_2_transfer: ", origin_2_transfer.value)
print("Optimal transfer_2_destination: ", transfer_2_destination.value)

Optimal value:  70500.00000747207
Optimal origin_2_transfer:  [[4.00000002e+02 1.99999998e+02 7.48803699e-08]
 [8.33124167e-07 1.20000000e+03 8.47329718e-08]
 [3.99999997e+02 3.46260093e-06 7.99999999e+02]]
Optimal transfer_2_destination:  [[3.05692052e-09 8.00000000e+02]
 [7.00000002e+02 6.99999998e+02]
 [7.99999998e+02 1.89050344e-06]]


In [11]:
print(f"Dual variable for supply constraints: {constraints[2].dual_value}")
print(f"Dual variable for capacity constraints: {constraints[3].dual_value}")
print(f"Dual variable for flow conservation constraints: {constraints[4].dual_value}")
print(f"Dual variable for demand constraints: {constraints[5].dual_value}")


Dual variable for supply constraints: [1.61712034e-09 4.00000000e+00 5.00000000e+00]
Dual variable for capacity constraints: [7.00000001e+00 1.42335066e-08 2.00000001e+00]
Dual variable for flow conservation constraints: [-10. -12. -15.]
Dual variable for demand constraints: [27.00000001 32.00000001]


The values of the dual variables in the problem have simple economic interpretations: 

1. The dual variables associated with the supply constraints are the shadow prices of the resources. They indicate the rate of change in the value of the optimal solution per unit increase in the amount of the resource available.
The dual variables associated with the capacity constraints are the shadow prices of the capacities. They indicate the rate of change in the value of the optimal solution per unit increase in the capacity of the constraint.

2. The dual variables associated with the demand constraints are the shadow prices of the demands. They indicate the rate of change in the value of the optimal solution per unit increase in the amount of the demand.

Because of this, I would invest in expanding the capacity of Jakarta, since it has the highest shadow price of the capacity/supply constraints. 

Because of the shadow prices of the demands, we also know that the shadow price of the demand for Donggala is the highest, so providing more resources to Donggala would be more expensive than providing more resources to Palu. 